# Carbon Emissions Data Pipeline: Step 1 - Data Loading
Copyright (C) 2022 OS-Climate

This notebook loads relevant source data to be used as source to the carbon dashboard:
* Last 12 hours of pod-level average hourly energy consumption for our data services, from kepler prometheus export
* Last 12 hours of carbon intensity data from ElectricityGrid CO2Signal API, from Kafka streaming service 

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [1]:
%%capture pipoutput

# For loading predefined environment variables from files
# Typically used to load sensitive access credentials
%pip install python-dotenv

# Interacting with Trino and using Trino with sqlalchemy
%pip install trino sqlalchemy sqlalchemy-trino

# Pandas and pyarrow
%pip install pandas
%pip install pyarrow
%pip install fastparquet 

# OS-Climate utilities to make data ingest easier
%pip install osc-ingest-tools

In [2]:
# Report exceptions only in cells output
%xmode Minimal

Exception reporting mode: Minimal


In [3]:
from dotenv import dotenv_values, load_dotenv
import os
import pathlib
import pandas as pd
import trino
from sqlalchemy.engine import create_engine
import osc_ingest_trino as osc

Define Environment and Execution Variables

In [4]:
# Load environment variables from credentials.env
osc.load_credentials_dotenv()

Connect to AWS Bucket for data loading

In [5]:
# Bucket must be configured with credentials for the Hive ingestion bucket
hive_bucket = osc.attach_s3_bucket('S3_HIVE')

In [6]:
# Define variable for Hive ingestion
hive_catalog = 'osc_datacommons_hive_ingest'
hive_schema = 'ingest'

Connect to Trino

In [7]:
# Define execution variables for loading to Iceberg / Trino
ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'sandbox'
ingest_table_pod_energy = 'dashboard_pod_energy'
ingest_table_carbon_intensity = 'dashboard_carbon_intensity'
ingest_bucket = 'osc-datacommons-s3-bucket-dev02'

In [8]:
engine = osc.attach_trino_engine(verbose=True, catalog=ingest_catalog)

using connect string: trino://caldeirav@trino-secure-odh-trino.apps.odh-cl2.apps.os-climate.org:443/osc_datacommons_dev


In [9]:
# Show available schemas to ensure trino connection is set correctly
schema_show_sql = f"""
show schemas in {ingest_catalog}
"""
schema_show = engine.execute(schema_show_sql)
print(schema_show.fetchall())

[('aicoe_osc_demo_results',), ('default',), ('demo_dv',), ('dera',), ('essd',), ('iceberg_demo',), ('information_schema',), ('ingest',), ('mdt_sandbox',), ('pcaf_sovereign_footprint',), ('sandbox',), ('wri_gppd',)]


Load pod-level average hourly energy consumption for our data services, from kepler prometheus export

In [10]:
sql=f"""
SELECT 
	element_at(labels, 'cluster') as cluster,
	element_at(labels, 'container') as container,
	element_at(labels, 'pod_namespace') as pod_namespace,
	element_at(labels, 'pod') as pod,
	element_at(labels, '__name__') as name,
	element_at(labels, 'service') as service,
	element_at(labels, 'namespace') as namespace,
	element_at(labels, 'job') as job,
	element_at(labels, 'pod_name') as pod_name,
	element_at(labels, 'instance') as instance,
	MONTH(timestamp) as month,
	DAY(timestamp) as day,
	HOUR(timestamp) as hour,
	AVG(value) as avg_pod_energy
FROM uwm_prometheus."default".pod_energy_total
WHERE timestamp > (NOW() - INTERVAL '12' hour)
	AND element_at(labels, 'pod_namespace') IN ('openmetadata', 'fybrik-system', 'odh-trino', 'inception', 'pachyderm', 'kafka', 'odh-superset', 'odh-jupyterhub')
GROUP BY element_at(labels, 'cluster'), element_at(labels, 'container'), element_at(labels, 'pod_namespace'), element_at(labels, 'pod'), element_at(labels, '__name__'), element_at(labels, 'service'), element_at(labels, 'namespace'), element_at(labels, 'job'), element_at(labels, 'pod_name'), element_at(labels, 'instance'), MONTH(timestamp), DAY(timestamp), HOUR(timestamp)
"""

In [11]:
df = pd.read_sql(sql, engine).convert_dtypes()

In [15]:
df.head()

,cluster,container,pod_namespace,pod,name,service,namespace,job,pod_name,instance,month,day,hour,avg_pod_energy
0,osc/osc-cl2,kepler-exporter,openmetadata,kepler-exporter-5hqdq,pod_energy_total,kepler-exporter,kepler-monitoring,kepler-exporter,airflow-web-7c56f65dff-fzfgm,ip-10-0-153-216.ec2.internal,10,14,23,217.0
1,osc/osc-cl2,kepler-exporter,odh-jupyterhub,kepler-exporter-5hqdq,pod_energy_total,kepler-exporter,kepler-monitoring,kepler-exporter,jupyterhub-2-lfl9g,ip-10-0-153-216.ec2.internal,10,14,23,97474086.0
2,osc/osc-cl2,kepler-exporter,openmetadata,kepler-exporter-5hqdq,pod_energy_total,kepler-exporter,kepler-monitoring,kepler-exporter,metadataessdingestiontask-01dccd85466b4b618bc8...,ip-10-0-153-216.ec2.internal,10,14,23,0.0
3,osc/osc-cl2,kepler-exporter,fybrik-system,kepler-exporter-5hqdq,pod_energy_total,kepler-exporter,kepler-monitoring,kepler-exporter,opa-58bbf7b9c7-s4grb,ip-10-0-153-216.ec2.internal,10,14,23,22390.0
4,osc/osc-cl2,kepler-exporter,fybrik-system,kepler-exporter-5hqdq,pod_energy_total,kepler-exporter,kepler-monitoring,kepler-exporter,opa-connector-5df9964998-xwspd,ip-10-0-153-216.ec2.internal,10,14,23,1582.0


Ingest pod-level average hourly energy consumption for our data services

In [14]:
# Perform ingestion of pandas dataframe
df.to_sql(ingest_table_pod_energy,
           con=engine,
           schema=ingest_schema,
           if_exists='append',
           index=False,
           method=osc.TrinoBatchInsert(batch_size = 100, verbose = False))

Load carbon intensity data from ElectricityGrid CO2Signal API, from Kafka streaming service

In [16]:
sql="""
SELECT _key as region, MONTH("_timestamp") as month, DAY("_timestamp") as day, HOUR("_timestamp") as hour, carbon_intensity, fossel_fuel_percentage, unit_value
FROM kafka_osclimate.electricitymap.co2signal
WHERE "_timestamp" > (NOW() - INTERVAL '24' hour)
ORDER BY _key, MONTH("_timestamp"), DAY("_timestamp"), HOUR("_timestamp")
"""

In [17]:
df_co2signal = pd.read_sql(sql, engine).convert_dtypes()

In [18]:
df_co2signal.head()

,region,month,day,hour,carbon_intensity,fossel_fuel_percentage,unit_value
0,US-CAL-BANC,10,14,10,447,90.03,gCO2eq/kWh
1,US-CAL-BANC,10,14,11,443,89.2,gCO2eq/kWh
2,US-CAL-BANC,10,14,12,443,89.2,gCO2eq/kWh
3,US-CAL-BANC,10,14,13,441,88.69,gCO2eq/kWh
4,US-CAL-BANC,10,14,14,433,87.01,gCO2eq/kWh


Ingest carbon intensity data from ElectricityGrid CO2Signal API

In [19]:
# Perform ingestion of pandas dataframe
df_co2signal.to_sql(ingest_table_carbon_intensity,
           con=engine,
           schema=ingest_schema,
           if_exists='append',
           index=False,
           method=osc.TrinoBatchInsert(batch_size = 100, verbose = False))